# Setup

See:
https://github.com/karpathy/arxiv-sanity-preserver

Package does much we need, 
but we would lack internal understanding,
until we break it down.

In [91]:
import arxiv
import json
import pandas as pd

from os import listdir
from os.path import isfile, join

In [82]:
# Paths
DATA_PATH = '../data/'
OUTPUT_PATH = '../data/raw/'
CREDS_PATH = '../collection/credentials/'

# Basic search

In [26]:
for key in search.__dict__.keys():
      print(key)

query
id_list
max_results
sort_by
sort_order


In [86]:
QUERY = "disinformation"
N =  2000

In [87]:
# sort_by : relevance, lastUpdatedDate, or submittedDate
# max_results : 2000 is limit irrc

search = arxiv.Search(
  query = QUERY,
  max_results = N, 
  sort_by = arxiv.SortCriterion.SubmittedDate 
)


In [88]:
# Quick results check, can still be long to run with many results

for result in search.results():
  print(result.title + '\n')

Faking Fake News for Real Fake News Detection: Propaganda-loaded Training Data Generation

Online User Profiling to Detect Social Bots on Twitter

DISCO: Comprehensive and Explainable Disinformation Detection

Automated clustering of COVID-19 anti-vaccine discourse on Twitter

Adversarial Robustness of Neural-Statistical Features in Detection of Generative Transformers

Fine-Grained Prediction of Political Leaning on Social Media with Unsupervised Deep Learning

Synthetic Disinformation Attacks on Automated Fact Verification Systems

GAN-generated Faces Detection: A Survey and New Perspectives (2022)

Outside Looking In: Approaches to Content Moderation in End-to-End Encrypted Systems

Redactor: Targeted Disinformation Generation using Probabilistic Decision Boundaries

UofA-Truth at Factify 2022 : Transformer And Transfer Learning Based Multi-Modal Fact-Checking

Team Yao at Factify 2022: Utilizing Pre-trained Models and Co-attention Networks for Multi-Modal Fact Verification

Reputat

In [ ]:
# get all data, not recommended

for result in search.results():     
      for key, value in result._raw.items(): #__dict__.items(): ### this is usual way, but here dict has raw
            print(key)
            print(value)
            print('\n')

In [89]:
# Save data to 

# overwrite 'w' or append 'a'
action = 'a'

#save as 
save_as = 'arxiv_' + QUERY

for result in search.results():
      with open (OUTPUT_PATH + save_as + '.jsonl', action) as f:
            json.dump(result._raw, f, default=str) # use raw as __dict__ has raw in it, thus more data
            f.write('\n')  

# Start here for loading existing data

In [92]:
# Check files in data folder
datafiles = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]
print(datafiles)

['arxiv_disinformation.jsonl']


In [93]:
# Load df
load_file = datafiles[0]

df = pd.read_json(DATA_PATH + load_file, convert_dates=True, lines=True, orient='records')

In [94]:
df.head(5)

,id,guidislink,link,updated,updated_parsed,published,published_parsed,title,title_detail,summary,...,authors,author_detail,author,links,arxiv_primary_category,tags,arxiv_comment,arxiv_doi,arxiv_journal_ref,arxiv_affiliation
0,http://arxiv.org/abs/2203.05386v1,True,http://arxiv.org/abs/2203.05386v1,2022-03-10T14:24:19Z,"[2022, 3, 10, 14, 24, 19, 3, 69, 0]",2022-03-10T14:24:19Z,"[2022, 3, 10, 14, 24, 19, 3, 69, 0]",Faking Fake News for Real Fake News Detection:...,"{'type': 'text/plain', 'language': None, 'base...",While there has been a lot of research and man...,...,"[{'name': 'Kung-Hsiang Huang'}, {'name': 'Kath...",{'name': 'Heng Ji'},Heng Ji,"[{'href': 'http://arxiv.org/abs/2203.05386v1',...","{'term': 'cs.CL', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",NaN,NaN,NaN,NaN
1,http://arxiv.org/abs/2203.05966v1,True,http://arxiv.org/abs/2203.05966v1,2022-03-09T22:42:29Z,"[2022, 3, 9, 22, 42, 29, 2, 68, 0]",2022-03-09T22:42:29Z,"[2022, 3, 9, 22, 42, 29, 2, 68, 0]",Online User Profiling to Detect Social Bots on...,"{'type': 'text/plain', 'language': None, 'base...",Social media platforms can expose influential ...,...,"[{'name': 'Maryam Heidari'}, {'name': 'James H...",{'name': 'Ozlem Uzuner'},Ozlem Uzuner,"[{'href': 'http://arxiv.org/abs/2203.05966v1',...","{'term': 'cs.LG', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",NaN,NaN,NaN,NaN
2,http://arxiv.org/abs/2203.04928v1,True,http://arxiv.org/abs/2203.04928v1,2022-03-09T18:17:25Z,"[2022, 3, 9, 18, 17, 25, 2, 68, 0]",2022-03-09T18:17:25Z,"[2022, 3, 9, 18, 17, 25, 2, 68, 0]",DISCO: Comprehensive and Explainable Disinform...,"{'type': 'text/plain', 'language': None, 'base...",Disinformation refers to false information del...,...,"[{'name': 'Dongqi Fu'}, {'name': 'Yikun Ban'},...",{'name': 'Jingrui He'},Jingrui He,"[{'href': 'http://arxiv.org/abs/2203.04928v1',...","{'term': 'cs.LG', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",NaN,NaN,NaN,NaN
3,http://arxiv.org/abs/2203.01549v1,True,http://arxiv.org/abs/2203.01549v1,2022-03-03T07:38:31Z,"[2022, 3, 3, 7, 38, 31, 3, 62, 0]",2022-03-03T07:38:31Z,"[2022, 3, 3, 7, 38, 31, 3, 62, 0]",Automated clustering of COVID-19 anti-vaccine ...,"{'type': 'text/plain', 'language': None, 'base...",Attitudes about vaccination have become more p...,...,"[{'name': 'Ignacio Ojea Quintana'}, {'name': '...",{'name': 'Colin Klein'},Colin Klein,"[{'href': 'http://arxiv.org/abs/2203.01549v1',...","{'term': 'cs.SI', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.SI', 'scheme': 'http://arxiv.org...","9 pages, 2 figures",NaN,NaN,NaN
4,http://arxiv.org/abs/2203.07983v1,True,http://arxiv.org/abs/2203.07983v1,2022-03-02T16:46:39Z,"[2022, 3, 2, 16, 46, 39, 2, 61, 0]",2022-03-02T16:46:39Z,"[2022, 3, 2, 16, 46, 39, 2, 61, 0]",Adversarial Robustness of Neural-Statistical F...,"{'type': 'text/plain', 'language': None, 'base...",The detection of computer-generated text is an...,...,"[{'name': 'Evan Crothers'}, {'name': 'Nathalie...",{'name': 'Paula Branco'},Paula Branco,"[{'href': 'http://arxiv.org/abs/2203.07983v1',...","{'term': 'cs.CL', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",NaN,NaN,NaN,NaN
